# ARPO Smoke Test

4 tasks, 2 envs, ~1 hour

**All dependencies explicitly installed**

In [ ]:
# Clone
from getpass import getpass
token = getpass('GitHub token: ')
!git clone https://{token}@github.com/gowathena/arpo_replica.git
%cd arpo_replica
!git checkout arpo-cpu-replicate
!git submodule update --init --recursive

In [ ]:
# Install ALL dependencies explicitly
!pip install -q transformers accelerate
!pip install -q ray omegaconf wandb
!pip install -q tensordict codetiming
!pip install -q mathruler
!pip install -q pylatexenc
!pip install -q qwen-vl-utils pillow

%cd OSWorld
!pip install -q -e .
%cd ..

import sys
sys.path.insert(0, '/content/arpo_replica')

print('✅ All deps installed')

In [ ]:
# Setup
!sudo service docker start
!docker pull happysixd/osworld-docker:latest

import ray
ray.init(num_cpus=4, num_gpus=1, ignore_reinit_error=True)

import wandb, os
from getpass import getpass
os.environ['WANDB_API_KEY'] = getpass('wandb key: ')
wandb.login()

!sed -i 's/vmware/docker/g' OSWorld/run_uitars.py
!sed -i 's/vmware/docker/g' OSWorld/run_multienv_uitars.py

print('✅ Ready!')

In [ ]:
# Config
import yaml

config = {
    'data': {'train_files': 'test_data/osworld_examples/train_smoke_4.json'},
    'algorithm': {'adv_estimator': 'grpo', 'disable_kl': True, 'enable_replay': True},
    'worker': {'actor': {'model': {'model_path': 'ByteDance-Seed/UI-TARS-2B-SFT'}, 'optim': {'lr': 1e-6}}},
    'env': {'num_envs': 2, 'max_steps': 16, 'provider': 'docker'},
    'trainer': {'total_episodes': 1, 'logger': ['console', 'wandb']},
}

with open('smoke.yaml', 'w') as f:
    yaml.dump(config, f)

print('✅ Config ready: 4 tasks, 2 envs, 16 steps')

In [ ]:
# Run
!python -m verl.trainer.main config=smoke.yaml